In [229]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import squarify
import ipywidgets as widgets
from IPython.display import display
from matplotlib.ticker import FuncFormatter

In [29]:
N = 5e5
df = pd.read_csv('instacart-data/order_products__prior.csv',chunksize=N, usecols=['product_id'])
df_product = pd.read_csv('instacart-data/products.csv', usecols=['product_id', 'department_id', 'aisle_id'])
df_department = pd.read_csv('instacart-data/departments.csv')
df_aisle = pd.read_csv('instacart-data/aisles.csv')

In [12]:
chunk = next(df)
chunk.head()

,product_id
500000,8193
500001,22395
500002,18394
500003,4920
500004,10246


In [13]:
df_product.head()

,product_id,aisle_id,department_id
0,1,61,19
1,2,104,13
2,3,94,7
3,4,38,1
4,5,5,13


In [20]:
df_product.groupby('department_id').get_group(1)

,product_id,aisle_id,department_id
3,4,38,1
7,8,116,1
11,12,119,1
17,18,79,1
29,30,38,1
...,...,...,...
49672,49673,129,1
49673,49674,37,1
49677,49678,129,1
49680,49681,38,1


In [30]:
def select_aisle_ID(df):
    order_aisle = pd.merge(df, df_product, on=['product_id'])[['department_id', 'aisle_id']]
    aisle_count = order_aisle.value_counts().reset_index()
    aisle_count.columns = ['department_id', 'aisle_id', 'counts']
    return aisle_count

chunk = next(df)
total_aisle_count = select_aisle_ID(chunk)

for i, chunk in enumerate(df):                    
    chunk_count = select_aisle_ID(chunk)
    total_aisle_count = pd.concat([total_aisle_count, chunk_count])

In [34]:
total_aisle_count = total_aisle_count.groupby(['department_id', 'aisle_id']).sum().reset_index()
total_aisle_count

,department_id,aisle_id,counts
0,1,34,71742
1,1,37,498425
2,1,38,390299
3,1,42,99369
4,1,52,232763
...,...,...,...
129,20,13,99032
130,20,14,129474
131,20,67,355685
132,20,96,395130


In [35]:
total_aisle_count = pd.merge(total_aisle_count, df_department, on='department_id')
total_aisle_count = pd.merge(total_aisle_count, df_aisle, on='aisle_id')
total_aisle_count

,department_id,aisle_id,counts,department,aisle
0,1,34,71742,frozen,frozen meat seafood
1,1,37,498425,frozen,ice cream ice
2,1,38,390299,frozen,frozen meals
3,1,42,99369,frozen,frozen vegan vegetarian
4,1,52,232763,frozen,frozen breakfast
...,...,...,...,...,...
129,20,13,99032,deli,prepared meals
130,20,14,129474,deli,tofu meat alternatives
131,20,67,355685,deli,fresh dips tapenades
132,20,96,395130,deli,lunch meat


In [36]:
total_aisle_count.drop(['department_id', 'aisle_id'], axis=1, inplace=True)

In [37]:
total_aisle_count

,counts,department,aisle
0,71742,frozen,frozen meat seafood
1,498425,frozen,ice cream ice
2,390299,frozen,frozen meals
3,99369,frozen,frozen vegan vegetarian
4,232763,frozen,frozen breakfast
...,...,...,...
129,99032,deli,prepared meals
130,129474,deli,tofu meat alternatives
131,355685,deli,fresh dips tapenades
132,395130,deli,lunch meat


In [40]:
total_aisle_group = total_aisle_count.groupby('department')

In [75]:
[*total_aisle_group.groups.keys()]

['alcohol',
 'babies',
 'bakery',
 'beverages',
 'breakfast',
 'bulk',
 'canned goods',
 'dairy eggs',
 'deli',
 'dry goods pasta',
 'frozen',
 'household',
 'international',
 'meat seafood',
 'missing',
 'other',
 'pantry',
 'personal care',
 'pets',
 'produce',
 'snacks']

In [222]:
def aisle_by_dep(dep_name='bakery'):
    data = total_aisle_group.get_group(dep_name)
    ax = data.plot.bar(x='aisle', y='counts', legend=False, color=sns.color_palette('bright'))
    ax.set_ylabel('counts', fontsize = 11, fontweight='black')
    ax.set_xlabel('')
    ax.set_title(dep_name, fontweight='black', fontsize=15)

    g = ax.get_yticks()
    ax.set_yticks(g[1:])
    ax.get_yaxis().set_major_formatter(
        FuncFormatter(lambda x,p: format(str(int(x/1000))+'K'))
    )
    plt.xticks(rotation=-30, ha='left')

In [221]:
w = widgets.interactive(aisle_by_dep, dep_name=[*total_aisle_group.groups.keys()])
widgets.interact_manual(aisle_by_dep, dep_name=[*total_aisle_group.groups.keys()])

interactive(children=(Dropdown(description='dep_name', index=2, options=('alcohol', 'babies', 'bakery', 'bever…

<function __main__.aisle_by_dep(dep_name='bakery')>

In [253]:
total_dep_count = total_aisle_group.sum()
total_dep_count

,counts
department,
alcohol,153696
babies,423802
bakery,1176787
beverages,2690129
breakfast,709569
bulk,34573
canned goods,1068058
dairy eggs,5414016
deli,1051249


In [274]:
list(total_dep_count.nlargest(10, columns='counts').index)

['produce',
 'dairy eggs',
 'snacks',
 'beverages',
 'frozen',
 'pantry',
 'bakery',
 'canned goods',
 'deli',
 'dry goods pasta']

In [283]:
[*total_dep_count.nlargest(10, columns='counts')['counts'], 123]

[9479291,
 5414016,
 2887550,
 2690129,
 2236432,
 1875577,
 1176787,
 1068058,
 1051249,
 866627,
 123]

In [286]:
def dep_nlargest(N=10, pallete='viridis'):
    top_dep = total_dep_count.nlargest(N, columns='counts')
    others_sum = total_dep_count['counts'].sum() - top_dep['counts'].sum()

    plt.figure(figsize=(12,5))
    squarify.plot(sizes=[*top_dep['counts'], others_sum], label=[*top_dep.index, 'others'],color=sns.color_palette(pallete))

In [287]:
g = widgets.interact_manual(dep_nlargest, N=(5, 18), pallete=['viridis', 'bright', 'deep'])

interactive(children=(IntSlider(value=10, description='N', max=18, min=5), Dropdown(description='pallete', opt…